### Install packages

### Import libraries

In [ ]:
import string
import matplotlib
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from nltk import word_tokenize

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import yaml

### Get data from GitHub

In [ ]:
csv_url = 'https://raw.githubusercontent.com/davisjrule/SQuAD-bot/main/train-squad.csv'
df = pd.read_csv(csv_url)
df.drop('id', axis=1, inplace=True)
df.drop('answer_start', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('context', axis=1, inplace=True)

df.head()

#df.iloc[1]

,question,text
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s


In [ ]:
with open('ai.yml', 'r') as file:
    data = yaml.safe_load(file)

df = pd.DataFrame(data = data['conversations'])
df

### Process data

In [ ]:
def clean_text(text):
  text = str(text).lower()
  text = text.replace("can't", "cannot")
  text = text.replace("won't", "will not")
  text = text.replace("i'm", "i am")
  text = text.replace("'ve", " have")
  text = text.replace("'ll", " will")
  text = text.replace("n't", " not")
  text = text.replace("'d", " would")
  text = text.replace("'s", " is")
  text = text.replace("'re", " are")
  for char in string.punctuation:
    if char != '-':
      text = text.replace(char, '')
  return text

In [ ]:
pairs = list(zip(df['question'],df['text']))
# pairs


In [ ]:
import numpy as np
import re

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

for line in pairs[:6000]:
  input_doc, target_doc = line[0], line[1]

  input_doc = clean_text(input_doc)
  target_doc = clean_text(target_doc)

  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))

  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
       
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)


input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

# target_tokens

In [ ]:
print(input_tokens)

['-', '00', '007', '05', '06', '08', '1', '10', '100', '1000', '10000', '100000000', '100m', '10217', '107', '11', '110th', '12', '120000', '121', '1260', '1294', '13', '14', '146', '15', '1573', '158', '1587', '16', '1600s', '1609', '1616', '1653', '1664', '1673', '1674', '17', '1700', '1700s', '1713', '1730', '1765', '1776', '1785', '1790', '18', '1806', '1811', '1817', '1818', '1820s', '1824', '1825', '1826', '1827', '1828', '1829', '1830', '1831', '1832', '1833', '1834', '1835', '1836', '1837', '1838', '1839', '1840', '1841', '1842', '1843', '1844', '1845', '1846', '1847', '1848', '1849', '1855', '1857', '1860', '1860s', '1861', '1863', '1865', '1870', '1880s', '1890', '1890s', '1892', '1897', '18th', '19', '1900', '1909', '1910', '1911', '1915', '1920s', '1924', '1928', '1930s', '1937', '1940', '1940s', '1942', '1945', '1950s', '1952', '1957', '1961', '1962', '1965', '1966', '1968', '1970s', '1980s', '1981', '1987', '1990', '1990s', '1991', '1992', '1993', '1995', '1997', '1999', 

In [ ]:
input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.



In [ ]:
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length), dtype='int32'
)
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length), dtype='int32'
)
decoder_output_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='int32'
  )
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    encoder_input_data[line, timestep] = input_features_dict[token]
  for timestep, token in enumerate(target_doc.split()):
    decoder_input_data[line, timestep] = target_features_dict[token]
    if timestep > 0:
        decoder_output_data[line, timestep - 1, target_features_dict[token]] = 1.


In [ ]:
from tensorflow.keras import layers , activations , models , preprocessing

In [ ]:
import tensorflow as tf

encoder_inputs = tf.keras.layers.Input(shape=( max_encoder_seq_length, ))
encoder_embedding = tf.keras.layers.Embedding( num_encoder_tokens, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_decoder_tokens, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_decoder_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 30, 200)      1285400     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 200)    984200      ['input_2[0][0]']                
                                                                                              

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=500 ) 
model.save( 'model.h5' ) 

Epoch 1/500
120/120 [==============================] - 37s 222ms/step - loss: 5.0124
Epoch 2/500
120/120 [==============================] - 16s 135ms/step - loss: 4.6280
Epoch 3/500
120/120 [==============================] - 14s 119ms/step - loss: 4.6007
Epoch 4/500
120/120 [==============================] - 15s 125ms/step - loss: 4.5664
Epoch 5/500
120/120 [==============================] - 14s 117ms/step - loss: 4.5376
Epoch 6/500
120/120 [==============================] - 14s 115ms/step - loss: 4.5061
Epoch 7/500
120/120 [==============================] - 14s 118ms/step - loss: 4.4792
Epoch 8/500
120/120 [==============================] - 14s 119ms/step - loss: 4.4585
Epoch 9/500
120/120 [==============================] - 15s 123ms/step - loss: 4.4320
Epoch 10/500
120/120 [==============================] - 14s 112ms/step - loss: 4.4008
Epoch 11/500
120/120 [==============================] - 14s 116ms/step - loss: 4.3625
Epoch 12/500
120/120 [==============================] - 13s 111

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [ ]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_features_dict[word] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_encoder_seq_length , padding='post')

In [ ]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = target_features_dict['<START>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = reverse_target_features_dict[sampled_word_index]
        decoded_translation += " " + sampled_word
        
        if sampled_word == '<END>' or len(decoded_translation.split()) > max_decoder_seq_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


In [ ]:
from keras.models import load_model
training_model = load_model('model.h5')

encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

def decode_response(test_input):
  states_values = encoder_model.predict(test_input)

  target_seq = np.zeros( ( 1 , 1 ) )
  target_seq[0, 0] = target_features_dict['<START>']
  stop_condition = False
  decoded_translation = ""
  while not stop_condition :
      dec_outputs , h , c = training_model.predict([ target_seq ] + states_values )

      sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
      sampled_word = reverse_target_features_dict[sampled_word_index]
      decoded_translation += " " + sampled_word
      
      if sampled_word == '<END>' or len(decoded_translation.split()) > max_decoder_seq_length:
          stop_condition = True
          
      empty_target_seq = np.zeros( ( 1 , 1 ) )  
      empty_target_seq[ 0 , 0 ] = sampled_word_index
      states_values = [ h , c ] 

  return decoded_translation


In [ ]:
def generate_response(input):
  input = clean_text(input)
  #tokens = re.findall(r"[\w']+|[^\s\w]", input)
  #input_matrix = np.zeros(
  #    (1, max_encoder_seq_length),
  #    dtype='int32')
  #for timestep, token in enumerate(tokens):
  #  if token in input_features_dict:
  #    input_matrix[0, timestep] = input_features_dict[token]
  #output = decode_response(input_matrix)
  output = decode_response(str_to_tokens(input))
  return output.replace("<START>","").replace("<END>","")


while True:
  reply = input()
  if reply == "STOP":
    break
  print(generate_response(reply))

In [ ]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

Legacy Code:

In [ ]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model

#Dimensionality
dimensionality = 256

#The batch size and number of epochs
batch_size = 10
epochs = 10

#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]

#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')

#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')
     

In [ ]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
  #Getting the output states to pass into the decoder
  states_value = encoder_model.predict(test_input)
  
  #Generating empty target sequence of length 1
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  
  #Setting the first token of target sequence with the start token
  target_seq[0, 0, target_features_dict['<START>']] = 1.
  
  #A variable to store our response word by word
  decoded_sentence = ''
    
  stop_condition = False

  while not stop_condition:
    #Predicting output tokens with probabilities and states
    output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    
    #Choosing the one with highest probability
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_token = reverse_target_features_dict[sampled_token_index]
    decoded_sentence += " " + sampled_token
    
    #Stop if hit max length or found the stop token
    if (sampled_token == '' or len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True
    
    #Update the target sequence
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.
    
    #Update states
    states_value = [hidden_state, cell_state]
  return decoded_sentence
     


In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
  
  #Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
  
  #Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix

  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)  
    chatbot_response = decode_response(input_matrix)
    #Remove  and  tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
  
  #Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

In [ ]:
def generate_response(input):
  input = clean_text(input)
  tokens = re.findall(r"[\w']+|[^\s\w]", input)
  input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
  for timestep, token in enumerate(tokens):
    if token in input_features_dict:
      input_matrix[0, timestep, input_features_dict[token]] = 1
  output = decode_response(input_matrix)
  return output.replace("<START>","").replace("<END>","")


while True:
  reply = input()
  if reply == "STOP":
    break
  print(generate_response(reply)+'\n')

In [ ]:
def save_model(model):
  model.save('')
  return pick_model

In [ ]:
def load_model():
  return model